# word2vec 개선

## Simple CBOW의 문제점

- window size를 1로 한정했음
    - 원하는 window size에 대해 계산할 수 있도록 개선 
<br/><br/>
- 어휘수가 많아질수록 계산량이 매우 커짐
- 특히 두가지 과정이 bottleneck
    1. input(one-hot vector)와 weight(W_in)의 곱셈
    2. hidden layer 에서 output으로 갈 때 weight(W_out)의 곱셈, softmax 계산
<br/><br/>
- 해결 방안
    1. Embedding layer를 통해 계산량 감소
    2. Negative sampling을 통해 loss 계산 과정 개선

## Embedding layer

- one-hot vector와 W_in을 곱하는 과정은 W_in의 특정 row를 추출하는 것과 같음
<br/><img src='../figs/fig%204-3.png' width='500px'>
<br/>
- matrix 연산을 하지 않고 row vector를 추출하는 `Embedding layer`를 통해 계산량 개선

In [1]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        output = W[self.idx]
        return output
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0 # 
#         # 좋지 않은 예. idx의 원소가 중복인 경우 문제가 생김
#         dW[self.idx] = dout 
#         for i, word_id in enumerate(self.idx):
#             dW[word_id] += dout[i]
        np.add.at(dW, self.idx, dout)
        return None

## Multi-class -> binary

- 다중 분류(multi-class classification)을 이진 분류(binary classification)으로 근사하려는 것 
     - 해당 자리에 target이 나타날 확률 -> 해당 자리에 위치한 단어가 target인지 아닌지로 변환  
- W_out의 target에 해당하는 column만 추출 후 hidden layer 뉴런과 내적 (Matrix x vector를 vector간 내적으로 변환)
    - Multi-class 에서 Softmax -> cross-entropy loss를 사용했던 것에서
    - Binary의 경우 Sigmoid -> cross-entropy loss 사용 

- 이때 Sigmoid layer와 Cross-entropy를 섞어서 back-propagation시 미분값이 $y-t$로 간단함.
<br/> <img src='../figs/fig%204-10.png'>

### 전체 과정

<br/> <img src = '../figs/fig%204-12.png'>

In [2]:
# h와 W_out의 embedding vector의 dot 부분 구현
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx) # row 추출
        out = np.sum(target_W + h, axis=1)
        
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        # 
        dout = dout.reshape(dout.shape[0], 1)
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

## Negative sampling

- binary classification으로 변환함으로써 계산량을 줄였음.
- 하지만 정답(positive)에 대해서만 학습했고, 오답(negative)에 대해서는 학습을 안함
    - 정답을 입력했을때 출력이 1에 가까운 방향으로 학습했음.
    - 하지만 오답을 입력했을때 출력이 0에 가까워지도록 만들어줘야함.
- 모든 negative에 대해 학습을 하기에는 어휘 수가 너무 늘어남
    - 적은 수의 negative를 sampling해서 사용.

### Sampling
- 자주 등장하는 단어가 많이 sampling되도록 각 단어의 출현 횟수의 확률분포에 따라 단어를 sampling.
- negative를 적게 sampling하므로 자주 나오지 않는 단어에 대해 학습하는 것 보단 흔하게 나오는 단어를 사용하는 것이 나음

In [3]:
# Unigram : 하나의 연속된 단어
# Bigram : 두개의 연속된 단어
# Trigram : 세개의 연속된 단어

class UnigramSampler:
    '''
    corpus에서 target에 대한 negative sampling
    '''
    def __init__(self, corpus, power, sample_size):
        self.corpus = corpus
        self.power = power
        self.sample_size = sample_size
        
    def get_negative_sample(self, target):
        neg_sample = []
        for t in target:
            # words : target을 제외한 단어 집합
            words = list(range(max(corpus)+1))
            words.remove(t)
            # 확률분포 계산
            p = [list(corpus).count(word)/len(corpus) for word in words]
            new_p = np.power(p, power)
            new_p /= np.sum(new_p)
            neg = np.random.choice(words, sample_size, p = new_p, replace=False)
            neg_sample.append(neg)
        return neg_sample, new_p

In [4]:
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample, p = sampler.get_negative_sample(target)
print(negative_sample)
print(p)

[array([4, 0]), array([2, 1]), array([3, 1])]
[0.2781948  0.2781948  0.2781948  0.16541561]


### Negative Sampling

In [17]:
from common.layers import SigmoidWithLoss

class NegativeSamplingLoss:
    def __init__(self, W, corpus, power = 0.75, sample_size = 5):
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.sample_size = sample_size # 샘플링 횟수
        self.loss_layer = [SigmoidWithLoss() for _ in range (sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        self.params, self.grads = [], []
        
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, h, target):
        batch_size = target.shape[0]
        neg_sample = self.sampler.get_negative_sample(target)
        # positive
        score = self.embed_dot_layers[0].forward(h, target)
        pos_label = np.ones(batch_size).astype(int)
        loss = self.loss_layers[0].forward(score, pos_label)
        # negative
        neg_label = np.zeors(batcs_size).astype(int)
        for i in range(self.sample_size):
            neg_target = neg_sample[:, i]
            score = self.embed_dot_layers[i].forward(h, neg_target)
            loss = self.loss_layers[i].forward(score, neg_label)
            
        return loss
    
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backwawrd(dout)
            dh += l1.backward(dscore)
            
        return dh

In [6]:
import sys
sys.path.append('..')
import numpy as np
#from common.layers import Embedding


In [8]:
class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        # initialize weight
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        # layer 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.neg_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
        
        # layer, parameter 정리
        layers = self.in_layers + [self.neg_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        # 단어의 분산 표현 저장
        self.word_vecs = W_in
        
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h /= len(self.in_layers)
        loss = self.neg_loss.forward(h, target)
        return loss
            
    
    def backward(self, dout=1):
        dout = self.neg_loss.backward(dout)
        dout /= len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
            
        return None

In [11]:
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import create_contexts_target
from data import ptb

In [13]:
# hyperparameter
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# load data
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)

In [18]:
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

In [24]:
from common.util import preprocess, create_contexts_target, convert_one_hot


window_size = 1
hidden_size = 5
batch_size = 3
max_epoch = 1000

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)


In [32]:
vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)

In [33]:
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

In [34]:
Trainer.fit(contexts, target, max_epoch, batch_size, epoch_print=2)

AttributeError: 'numpy.ndarray' object has no attribute 'eval_interval'